In [1]:
import operator
import math

In [21]:
class Asteroid:
    '''
    Treat each asteroid as an object so we can
    store its coordinates and status centrally
    '''
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.status = True
        self.slope = False
        self.destroyed = False
        self.destroyed_order = -1
        
    def calculateSlope(self, mon):
        '''
        Calculating the slope in radians so we can determine
        which ones to hit.
        '''
        try:
            self.slope = math.atan2(mon.y - self.y, mon.x - self.x)
            #self.slope = ((mon.y - self.y) / (mon.x - self.x))
        except ZeroDivisionError:
            self.slope = "Infinite"
            
        # We need to adjust things so we sweep based on the order the puzzle wants
        # We want to go from pi/2 to pi
        # then -pi to 0, and then 0 to pi/2
        # There's probably a different way to do this that is easier, but...let's rotate the points' slopes
        if self.slope >= math.pi / 2 and self.slope <= math.pi:
            # Move this towards zero as they are first
            self.slope = self.slope - (math.pi / 2)
        elif self.slope >= -math.pi and self.slope <= math.pi / 2:
            # Make these much larger as they should happen after the first points.
            self.slope = self.slope + (math.pi * 2)
    
    def printPoint(self):
        return "({},{})".format(self.x, self.y)
            
        
            

class Monitor:
    '''
    The entire system we're checking
    '''
    def __init__(self, amap):
        print("[-] Setting up...")
        self.text_map = amap
        self.asteroid_map = self.processMap()
        self.mon = False
        self.mon_count = 0
        
    def calculateAsteroidSlopes(self):
        '''
        To do the sweeping motion, we need to get the radians of each
        point as it relates to the monitoring point.
        Our sweep will begin at 0, go to pi, flip negative, and go 
        back down to zero again.
        '''
        print("[-] Finding slopes to asteroids...")
        for item in self.asteroid_map:
            if self.isSame(item, self.mon):
                continue
            else:
                item.calculateSlope(self.mon)
                
        self.asteroid_map.sort(key=lambda x: x.slope)
                
    def printSlopes(self):
        '''
        A simple output function to throw out the slopes we found...
        '''
        for item in self.asteroid_map:
            if self.isSame(item, self.mon):
                continue
            else:
                print("({},{}) = {}".format(item.x, item.y, item.slope))
        
        
    def findBestMonitorLocation(self):
        '''
        This function finds the best monitor location.
        Was outside the class object last time in part 1...
        '''
        print("[-] Processing...")
        result = mon.testViewPoints()
        for item in result:
            if result[item]["count"] > self.mon_count:
                self.mon_count = result[item]["count"]
                self.mon = result[item]["point"]
        
        print("Monitor set at position ({},{})".format(self.mon.x, self.mon.y))
        self.calculateAsteroidSlopes()
        
    def processMap(self):
        '''
        This takes the string map given to us
        and turns it into something we can use
        '''
        asteroids = list()
        listmap = self.text_map.split('\n')
        
        for ridx, row in enumerate(listmap):
            for cidx, col in enumerate(row):
                if col == '#':
                    # Astroid found, create point and add to list
                    asteroids.append(Asteroid(cidx, ridx))
        return asteroids
    
    def checkView(self, mon_point):
        '''
        This looks over all the asteroids, and checks to see which
        ones are visible given the mon_point we're testing
        '''
        for end_point in self.asteroid_map:
            if end_point.destroyed is True:
                continue
            for mid_point in self.asteroid_map:
                if mid_point.destroyed is True:
                    continue
                elif self.isSame(end_point, mid_point) or self.isSame(mon_point, mid_point):
                    continue
                elif self.isOn(mon_point, end_point, mid_point):
                    end_point.status = False
    
    def resetView(self):
        '''
        After we check a monitoring point, we want to reset the view
        back to the original, ready for the next test
        '''
        for point in self.asteroid_map:
            point.status = True
    
    def countViewablePoints(self, mon_point):
        '''
        After the tests are done, count up the asteroids that are viewable
        '''
        count = 0
        self.checkView(mon_point)
        for point in self.asteroid_map:
            if not self.isSame(mon_point, point) and point.status is True:
                count += 1
        return count
    
    def testViewPoints(self):
        '''
        Main function tying things together and returning the result to the user
        '''
        result = dict()
        for mon_point in self.asteroid_map:
            count = self.countViewablePoints(mon_point)
            thiskey = "({},{})".format(mon_point.x, mon_point.y)
            result[thiskey] = {
                "count": count,
                "point": mon_point
            }
            self.resetView()
            
        
        return result
    
    def isSame(self, a, b):
        '''
        This function is used to try to avoid typing this same thing over
        and over. We don't want to check if a monitor can see itself!
        '''
        if a.x == b.x and a.y == b.y:
            return True
        else:
            return False
    
    
    # Point functions taken from here: https://stackoverflow.com/questions/328107/how-can-you-determine-a-point-is-between-two-other-points-on-a-line-segment 
    def isOn(self, a, b, c):
        '''
        Return true iff point c intersects the line segment from a to b.
        '''
        # (or the degenerate case that all 3 points are coincident)
        return (self.collinear(a, b, c)
                and (self.within(a.x, c.x, b.x) if a.x != b.x else 
                     self.within(a.y, c.y, b.y)))

    def collinear(self, a, b, c):
        '''
        Return true iff a, b, and c all lie on the same line.
        '''
        return (b.x - a.x) * (c.y - a.y) == (c.x - a.x) * (b.y - a.y)

    def within(self, p, q, r):
        '''
        Return true iff q is between p and r (inclusive).
        '''
        return p <= q <= r or r <= q <= p
    
    def laserSweep(self, target_count):
        '''
            We want to only hit asteroids that are visible at this monitoring point
            Reset view
            laser again.
            On top of this, we need to count the asteroids as they're being destroyed 
            so we know which one
        '''
        print("[-] Destroying!")
        # Resetting view to be that of the monitor's so we know which ones are visible
        self.checkView(self.mon)
        
        sweep_count = 0
        hit_count = 0
        
        while hit_count < len(self.asteroid_map) - 1:
            for item in self.asteroid_map:
                if item.status is True and item.destroyed is False:
                    print("{} is #{} Destroyed!".format(item.printPoint(), hit_count+1))
                    item.destroyed = True
                    item.destroyed_order = hit_count
                    hit_count += 1
                    if hit_count+1 == target_count:
                        print("***** ---> Target Destroyed!!! <--- *****")
            sweep_count += 1
            print("Sweep #{} Done. ({}/{} Hits) ".format(sweep_count, hit_count, len(self.asteroid_map)-1))
            self.resetView()
            self.checkView(self.mon)
        

In [22]:
test_map = '''.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....#...###..
..#.#.....#....##'''
mon = Monitor(test_map)
mon.findBestMonitorLocation()
mon.calculateAsteroidSlopes()
mon.laserSweep(30)

[-] Setting up...
[-] Processing...
Monitor set at position (8,3)
[-] Finding slopes to asteroids...
[-] Finding slopes to asteroids...
[-] Destroying!
(8,1) is #1 Destroyed!
(9,0) is #2 Destroyed!
(9,1) is #3 Destroyed!
(10,0) is #4 Destroyed!
(9,2) is #5 Destroyed!
(11,1) is #6 Destroyed!
(12,1) is #7 Destroyed!
(11,2) is #8 Destroyed!
(15,1) is #9 Destroyed!
(12,2) is #10 Destroyed!
(13,2) is #11 Destroyed!
(14,2) is #12 Destroyed!
(15,2) is #13 Destroyed!
(12,3) is #14 Destroyed!
(16,4) is #15 Destroyed!
(15,4) is #16 Destroyed!
(10,4) is #17 Destroyed!
(4,4) is #18 Destroyed!
(2,4) is #19 Destroyed!
(2,3) is #20 Destroyed!
(0,2) is #21 Destroyed!
(1,2) is #22 Destroyed!
(0,1) is #23 Destroyed!
(1,1) is #24 Destroyed!
(5,2) is #25 Destroyed!
(1,0) is #26 Destroyed!
(5,1) is #27 Destroyed!
(6,1) is #28 Destroyed!
(6,0) is #29 Destroyed!
***** ---> Target Destroyed!!! <--- *****
(7,0) is #30 Destroyed!
Sweep #1 Done. (30/36 Hits) 
(8,0) is #31 Destroyed!
(10,1) is #32 Destroyed!
(14,

In [23]:
with open('Day10-input.txt') as mapfile:
    real_map = mapfile.read()
mon = Monitor(real_map)
mon.findBestMonitorLocation()
mon.calculateAsteroidSlopes()
mon.laserSweep(200)


[-] Setting up...
[-] Processing...
Monitor set at position (20,21)
[-] Finding slopes to asteroids...
[-] Finding slopes to asteroids...
[-] Destroying!
(20,18) is #1 Destroyed!
(21,1) is #2 Destroyed!
(21,3) is #3 Destroyed!
(21,4) is #4 Destroyed!
(21,5) is #5 Destroyed!
(21,7) is #6 Destroyed!
(21,9) is #7 Destroyed!
(21,11) is #8 Destroyed!
(22,3) is #9 Destroyed!
(21,13) is #10 Destroyed!
(21,14) is #11 Destroyed!
(23,1) is #12 Destroyed!
(22,8) is #13 Destroyed!
(23,2) is #14 Destroyed!
(21,15) is #15 Destroyed!
(23,5) is #16 Destroyed!
(21,16) is #17 Destroyed!
(23,7) is #18 Destroyed!
(22,12) is #19 Destroyed!
(23,9) is #20 Destroyed!
(23,11) is #21 Destroyed!
(22,15) is #22 Destroyed!
(23,14) is #23 Destroyed!
(21,19) is #24 Destroyed!
(23,16) is #25 Destroyed!
(22,18) is #26 Destroyed!
(23,17) is #27 Destroyed!
(22,19) is #28 Destroyed!
(22,20) is #29 Destroyed!
(21,21) is #30 Destroyed!
(23,22) is #31 Destroyed!
(22,22) is #32 Destroyed!
(23,23) is #33 Destroyed!
(22,23) is